<a href="https://colab.research.google.com/github/ryouy/AKaggle/blob/main/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
competition_name = "spaceship-titanic"
from google.colab import userdata
import os
os.environ['KAGGLE_USERNAME'] = 'ryohmats'
os.environ['KAGGLE_KEY'] = userdata.get('kaggle')

!pip install kaggle
!kaggle competitions download -c {competition_name}
!unzip -q /content/{competition_name}.zip -d ./
!rm -rf /content/{competition_name}.zip
!rm -rf sample_data/

!pip install ydata-profiling
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_extracted = df_train.sample(n=1000, random_state=42)
profile = ProfileReport(df_extracted, title="Training Data Report", explorative=True)
profile.to_file("train_report_extracted.html")

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 68.8MB/s]
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=968fbdfd602899a0a49ec0cca8e08a4b75737c0dad989f2f8ab42c0861d6c142
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
   

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [270]:

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
data= pd.concat([df_train,df_test],axis=0,sort=False)
Id_data=pd.concat([df_train['PassengerId'],df_test['PassengerId']],axis=0,sort=False)
data=data.drop(['Name'],axis=1)

data['CryoSleep'].replace({True:1,False:0},inplace=True)
data['VIP'].replace({True:1,False:0},inplace=True)
data['Transported'].replace({True:1,False:0},inplace=True)
data['Destination'].replace({'TRAPPIST-1e':0,'55 Cancri e':1,'PSO J318.5-22':2},inplace=True)
data['HomePlanet'].replace({'Earth':0,'Europa':1,'Mars':2},inplace=True)
data=data.drop(['PassengerId'],axis=1)

data=data.drop(['Cabin'],axis=1)
df_train['Transported'].value_counts()

,count
Transported,
True,4378
False,4315


In [272]:
data.head(-1)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,1.0,0.0,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0
3,1.0,0.0,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0.0,0.0,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4271,0.0,0.0,0.0,40.0,0.0,0.0,865.0,0.0,3.0,0.0,NaN
4272,0.0,1.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4273,0.0,0.0,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0,NaN
4274,2.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [281]:
#data['Service'] = data['RoomService'] + data['FoodCourt'] + data['ShoppingMall'] + data['Spa'] + data['VRDeck']
#data=data.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'],axis=1)

# 各列の欠損値を埋める
for col in data.columns:
    if col == "Transported":  # Transported列を特別処理
        #data[col].fillna(-1, inplace=True)  # 欠損値を一時的に -1 に置き換え
        continue
    if col == "Destination":
      data['Destination'].fillna(data['Destination'].mode(), inplace=True)
    if data[col].dtype in ['float64', 'int64']:  # 数値型
        data[col].fillna(data[col].mean(), inplace=True)

data['VIP']=data['VIP']*data['VIP']
data['Destination']=data['Destination']*data['Destination']

# Transported列が0, 1, 欠損値のみになるように確認
#data['Transported'] = data['Transported'].astype(int)
data.head(-3)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,1.0,0.0,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0
3,1.0,0.0,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0.0,0.0,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4269,0.0,1.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4270,2.0,0.0,0.0,43.0,0.0,47.0,0.0,3851.0,0.0,0.0,NaN
4271,0.0,0.0,0.0,40.0,0.0,0.0,865.0,0.0,3.0,0.0,NaN
4272,0.0,1.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [282]:
df_train = data.iloc[:8693]  # 最初の8693行
df_test = data.iloc[8693:]
Id_data_train = Id_data.iloc[:8693]
Id_data_test = Id_data.iloc[8693:]

X_train = df_train.drop('Transported', axis=1)
y_train = df_train['Transported']
X_test = df_test.drop('Transported', axis=1)
y_test = df_test['Transported']

#from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
#clf.fit(X_train, y_train)
#y_pred = clf.predict(X_test)
import xgboost as xgb
from sklearn.metrics import accuracy_score

# XGBoostモデルを定義
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)

# モデルの学習
xgb_clf.fit(X_train, y_train)

# 予測
y_pred = xgb_clf.predict(X_test)

# 結果を表示
#print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred)}")
print(y_train.unique())  # ユニークな値を表示
print(y_test.unique())  # テストデータのターゲット値も確認

y_pred_df = pd.DataFrame(y_pred, columns=["Transported"])
y_pred_df = pd.concat([Id_data_test, y_pred_df], axis=1)
y_pred_df['Transported'] = y_pred_df['Transported'].replace({0: False, 1: True})


y_pred_df.to_csv('submission.csv', index=False)
print(y_pred_df['Transported'].value_counts())
print('###########')
print(2465, 1812)

[0. 1.]
[nan]
Transported
True     2271
False    2006
Name: count, dtype: int64
###########
2465 1812
